<b>1. Introduction</b>

The objective of this presentation is to propose a clustering of the client data, detailing the process and describing the weak and strong points.

<b>2. Clean Data & Feature Engineering</b>

The data set did not have missing values and each variable has consistent data types in all observations. 

However there were problems regarding AGE and PROFESSION such as "ANALISTA" or "ADMINISTRADOR" younger than ten years old. That data was removed of the analysis until further clarification.

Variables "PROFISSAO" E "ESTADO CIVIL" were simplified by taking the first word of the description.

The variable "GEO_REFERENCIA" was used to create new features, specifically, the raking of four clusters by percentile of each geo reference in relation to variables "VALOR{1,2,3,4}". 

<b>3. Variable importance</b>

Given the large number of variables, it is important to reduce the data to the most relevant ones. In order to do this, we select one variable (VALOR_1) as the target, and the remaining as the explanatory variables. We conclude that IDADE AND VALOR variables are the most important. The least important were specific professions with few data available.

<b>4. Clustering</b>

The clustering algorithm chosen is KMeans. This algorithm calculates the distance between data points to define groups. One problem in this application is that the grouping was very skewed with one group containg >90% of all users. A solution to this is removing outliers and normalizing data.

<b>5. Cross Validation</b>

Clustering is an unsupervised algorithm. However, is it easier to assess the performance of an approach with a target variable.

The typical method to test the performance of a model is to split the dataset in two different samples, in order to avoid information leakage, training the model in one and testing in the other one.

The metric of performance choosen was RMSE, because the target variable is continuous. This method calculates the root mean square error between the true values and the predicted values. The model with the lowest RMSE would be the best.

In [ ]:
import numpy as np
import pandas as pd
import datetime as datetime

from functions import *

import collections
from patsy import dmatrices

import sklearn as sk
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
import statsmodels.formula.api as smf

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print np.__version__  # 1.11.1
print pd.__version__  # 0.18.1
print sk.__version__  # 0.17.1
print sns.__version__ # 0.5.1

In [ ]:
local_path = !pwd

In [ ]:
df=pd.read_excel(local_path[0] + "/Dataset - Code Challenge Data Scientist.xlsx")

In [ ]:
# 1.No missing values : Check
df.shape == df.dropna().shape

In [ ]:
# 2. Each variable has only one data type : Check
for col in df.columns:
    print len(set(map(type, df[col].values))), " : ",set(map(type, df[col].values)), col 

## 2. Clean Data  & Feature Engineering

In [ ]:
df = clean_data(df)

## 3. Variable importance

In [ ]:
formula = "VALOR_01 ~ C(PERFIL) + IDADE + C(PROFISSAO_GERAL) + C(ESTADO_CIVIL_GERAL) + VALOR_02 + " \
"VALOR_03 + VALOR_04 + C(VALOR_01_GEO_REF) + C(VALOR_02_GEO_REF) + C(VALOR_03_GEO_REF) + C(VALOR_04_GEO_REF)"

In [ ]:
formula

In [ ]:
y, X = dmatrices(formula, data=df, return_type='dataframe')

In [ ]:
clf = RandomForestClassifier()
clf.fit(X,
        np.asarray(y['VALOR_01'], dtype="|S6"))

imp = clf.feature_importances_
names = X.columns

imp, names = zip(*sorted(zip(imp, names)))

imp_by_var = dict(zip(names, imp))

df_var_importance = pd.DataFrame(imp_by_var, index=[0]).transpose()
df_var_importance.columns = ["var_importance"]
df_var_importance = df_var_importance.sort_values(by="var_importance", ascending=False)
df_var_importance.head()

In [ ]:
df_var_importance.head(10).plot(kind="barh", title="Most Important Variables")

In [ ]:
df_var_importance.tail(10).plot(kind="barh", title="Least Important Variables")

## 4. Clustering

In [ ]:
formula = "ID ~ C(PERFIL) + IDADE + C(PROFISSAO_GERAL) + C(ESTADO_CIVIL_GERAL) + VALOR_01 + VALOR_02 + " \
"VALOR_03 + VALOR_04 + C(VALOR_01_GEO_REF) + C(VALOR_02_GEO_REF) + C(VALOR_03_GEO_REF) + C(VALOR_04_GEO_REF)"

y, X = dmatrices(formula, data=df, return_type='dataframe')

In [ ]:
# Cluster
km = KMeans(4, init='k-means++') # initialize
km.fit(X)
c = km.predict(X) # classify into three clusters
print collections.Counter(c)

In [ ]:
#df.PERFIL.nunique()
print collections.Counter(df.PERFIL.values)

In [ ]:
X["cluster"] = c

In [ ]:
sns.lmplot('VALOR_01', 'VALOR_02', data=X, hue='cluster', fit_reg=False)
plt.title("VALOR_01 VS VALOR_02")
plt.show()

In [ ]:
sns.lmplot('VALOR_02', 'VALOR_03', data=X, hue='cluster', fit_reg=False)
plt.title("VALOR_02 VS VALOR_03")
plt.show()

In [ ]:
sns.lmplot('VALOR_01', 'VALOR_03', data=X, hue='cluster', fit_reg=False)
plt.title("VALOR_01 VS VALOR_03")
plt.show()

## 5. Cross Validation

In [ ]:
model_formula = "VALOR_01 ~ C(PERFIL) + IDADE + C(ESTADO_CIVIL_GERAL) + VALOR_02 + " \
"VALOR_03 + VALOR_04 + C(VALOR_01_GEO_REF) + C(VALOR_02_GEO_REF) + C(VALOR_03_GEO_REF) + C(VALOR_04_GEO_REF)"

In [ ]:
train, test = cross_validation.train_test_split(df, test_size = 0.5)

In [ ]:
mod = smf.ols(formula=model_formula, data=train)
res = mod.fit()
print res.summary()

In [ ]:
y_true = test["VALOR_01"]
y_pred = res.predict(test)

In [ ]:
mean_squared_error(y_true, y_pred)